<a href="https://colab.research.google.com/github/lephuocdat2000/-CS114.K21-/blob/master/SarcasmDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Mô tả bài toán***: Đây là bài toán nhằm dự đoán xem một bài báo có phải là châm biếm hay không dựa vào tiêu đề của nó.

***Cách thức thu thập dataset:*** 
 - Thu thập các headline và đường link của dataset về châm biếm từ trang báo https://www.theonion.com/ , đây là trang chỉ viết các bài báo châm biếm.
 - Thu thập các headline và đường link của dataset không phải châm biếm từ trang báo https://www.huffpost.com/, đây là trang chỉ viết các bài báo chính thống. 

Đầu tiên tiến hành liên kết googledrive với googlecolab để lấy dữ liệu

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')  #liên kết nội dung của googledrive vào googlecolab tại thư mục có tên drive trên googlecolab

Tiến hành đọc dữ liệu bằng thư viện pandas và đường liên kết của file dữ liệu có sẵn trong thư mục theo đường link drive/My Drive/SarcasmDetection

Tiến hành đọc dữ liệu gốc của bài toán từ 2 file .json

In [ ]:
import pandas as pd
data1=pd.read_json('/content/drive/My Drive/SarcasmDetection/Sarcasm_Headlines_Dataset.json',lines=True) 
#lines=True ở đây có nghĩa là đọc dữ liệu theo từng dòng
data1

In [ ]:
data2=pd.read_json('/content/drive/My Drive/SarcasmDetection/Sarcasm_Headlines_Dataset_v2.json',lines=True)

In [ ]:
data2

Tiến hành ghép 2 bộ dữ liệu chúng ta vừa đọc thành một bộ dữ liệu tổng quát bằng câu lệnh pd.concat(câu lệnh ghép các đối tượng pandas với nhau theo trục)
ở đây cố định là trục 0 hay trục dọc nên k cần thay đổi

In [ ]:
data=pd.concat([data1,data2],ignore_index=True)   

ignore_index=True nghĩa là bộ data2 sẽ được gộp vào bộ data1 và sẽ nhận các giá trị index mới, không phải từ  1 -> 28618  mà bắt đầu từ 26709 -> 55327

In [ ]:
data

Bộ dữ liệu khá lớn, có 3 features: + article_link (đường dẫn)
                                  + headline ( tiêu đề bài báo)
                                  + is_sarcastic (nếu bằng 1 thì bài báo đó là sarcasm, bằng 0 thì ngược lại).
Tuy nhiên đối với bài toán này ta chỉ sử dụng headline và is_sarcastic

Tiến hành đọc bộ dữ liệu mới tự thu thập được lưu trên ggdrive
(link dataset ở cuối bài)

---



In [ ]:
collected_data=pd.read_excel('/content/drive/My Drive/SarcasmDetection/newdataset.xlsx')
collected_data

Bộ dữ liệu mới gồm 2036 bài báo trong đó gồm 1026 bài là nonsarcasm và 1010 bài là sarcasm

**Feature Engineering**

---



Kiểm tra xem có bao nhiêu kí tự số trong tập headlines

In [ ]:
numbers=0
c=''
for x in data['headline']:
   for c in x:
      if (c.isdigit()==True): numbers+=1
numbers

Ta thấy rằng số kí tự số rất lớn tuy nhiên đối với yêu cầu của bài toán này thì các kí tự số không phản ánh được vấn đề bài toán mà thậm chí còn gây nhiễu, sai lệch trong quá trình training => Ta tiến hành loại bỏ kí tự số

In [ ]:
X=data['headline'].str.replace('\d+', '')


 \d+: chuỗi kí tự có một hoặc nhiều chữ số, câu lệnh ở phía trên là để thay thế toàn bộ kí tự chữ số thành kí tự trống 


Tách bộ dữ liệu thành 2 phần training set và test set với tỉ lệ 80% và 20% => Tránh overfitting

In [ ]:
from sklearn.model_selection import train_test_split 
y=data['is_sarcastic']
X_train,X_validation,y_train,y_validation = train_test_split(X,y,test_size=0.2,random_state=1,) 

random_state=1 => lựa chọn một cách ngẫu nhiên 

Ta tiến hành xây dựng Bag of Words sau đó chuyển đổi các headlines về dạng ma trận với hàm CountVectorizer từ thư viện sklearn, tiếp đến dùng hàm TfidfTransformer để đánh giá lại về giá trị các từ bởi vì có những từ tuy xuất hiện nhiều lần nhưng nó lại thực sự không quan trọng (the, and, is,of,that,...)=> giúp model đạt kết quả chính xác hơn

VD để hiểu rõ hơn về 2 hàm CountVectorizer() và TfidfTransformer
 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
sentence=['This is the first document.',
          'This is the second second document.',
          'And the third one.',
          'Is this the first document?',]
vectorizer = CountVectorizer()
y = vectorizer.fit_transform(sentence)
print("Các từ trong đoạn văn:")
print(vectorizer.get_feature_names())
print("Ma trận hóa đoạn văn:")
print(y.toarray())

Cách thức hàm CountVectorizer ma trận hóa đoạn văn bản ví dụ ở trên:
    
   


- Đầu tiên liệt kê các loại từ có trong văn bản ( ở ví dụ trên là 9 loại từ ) hay tạo mảng gồm các loại từ, ở đây giả sử gọi là mảng k.

 - Tiến hành ma trận hóa văn bản thành ma trận 9 cột 4 hàng (văn bản gồm 4 dòng)

 Ví dụ về cách chuyển hàng đầu tiên: Xét các từ trong mảng k từ đầu mảng đến cuối mảng, bắt đầu với từ 'and' thì trong câu đầu tiên xuất hiện 0 lần => X[0][0]=0, từ 'document' xuất hiện 1 lần X[0][1]=1, từ 'first' xuất hiện 1 lần => X[0][2]=1,'is' xuất hiện 1 lần => X[0][3]=1, 'one' xh 0 lần => x[0][4] = 0, 'second' xuất hiện 0 lần => x[0][5]=0, 'the' xuất hiện 1 lần => x[0][6]=1,'third' xuất hiện 0 lần => x[0][7]=0, 'this' xuất hiện 1 lần => x[0][8]=1
Vậy ta được ma trận chuyển hóa với hàng đầu tiên là: 0 1 1 1 0 0 1 0 1 
Các hàng còn lại ta có thể làm tương tự để được kết quả như trên

Bằng việc chuyển đổi như trên thì ta được một ma trận số hóa của văn bản tuy nhiên như vậy thì những từ không có nhiều ý nghĩa như (the,and,is,..) lại có cùng mức độ quan trọng so với những từ như (document,..) => điều này có thể gây sai lệch cho model trong quá trình training nên ta phải đánh giá lại giá trị của các từ bằng TfidfTransformer()

In [ ]:
transformer = TfidfTransformer()
print(y.toarray())
tfidf = transformer.fit_transform(y).toarray()
print(tfidf)

Giải thích về cách tính Tfidf ở đây:
  
  
  


   * Tfidf = Tf * idf,với:

**-** Tf: tần số xuất hiện của một từ trong văn bản đó = Số lần xuất hiện của từ nhất định / Số lần xuất hiện nhiều nhất của một từ bất kỳ

 **-**idf: nghịch đảo tần số của văn bản chứa từ nhất định = log(Tổng số văn bản / số văn bản chứa từ đó+1)

Ta thấy những từ không quan trọng (the,and,it,is,...) thường xuất hiện nhiều trong các văn bản cho nên theo công thức trên ta thấy số văn bản chứa từ đó càng nhiều thì idf càng thấp => tfidf càng thấp => tầm quan trọng của từ đó thấp

**Qua ví dụ trên ta có thể hiểu rõ hơn về Ma trận hóa văn bản và đánh giá lại mức độ quan trọng của các từ. Vậy ta tiến hành tương tự với data gốc đã cho.**

***Xây dựng model***

Thay vì tự thực hiện tuần tực các bước biến đổi CountVectorizer(),TfidfTransformer() và fit model thì ta có thể xây dựng pipeline để liên kết 3 bước lại với nhau cho máy tự thức hiện một cách nhanh gọn và thuận lợi hơn trong việc tinh chỉnh tham số.

Vì đây là bài toán phân lớp nên ta sẽ chọn những thuật toán phân lớp. Thuật toán lựa chọn để train model ở đây là Multinomial Naive Bayes vì đây là thuật toán được sử dụng nhiều trong text classification mà các feature vectors được tính bằng Bag of words, nó thực hiện nhanh, hàm dự đoán xác suất đơn giản nên dễ giải thích, tinh chỉnh tham số. Nhưng lí do chính là ngoài linear regression thì em chỉ mới hiểu được mỗi giải thuật này

NAIVE BAYES


In [ ]:
from sklearn.naive_bayes import MultinomialNB # thuật toán MultinomialNaiveBayes
from sklearn.pipeline import Pipeline  #import pipeline => xây dựng pipeline cho model
from sklearn.metrics import accuracy_score #đánh giá hiệu suất model
target_name=['is_sacarsm','non_sacarsm']
#xây dựng pipeline theo tuần tự các bước như đã nêu ở trên
model=Pipeline([('vect',CountVectorizer()),     #stop_word='english' => loại bỏ các từ thuộc stop_word trong tiếng Anh ( the,on,a,an,...) 
               ('tfidf',TfidfTransformer(False)),
               ('clf',MultinomialNB()),
])
model.fit(X_train,y_train)

Quá trình học của model: model sẽ tính toán xác xuất có điều kiện của mỗi word xuất hiện trên nhãn của nó (is_sarcasm or sarcasm) với công thức: 

* P(w|label) = (count(w|label)+1) /(count(c)+|V|) 

         * count(w/label): số lượng w có trong văn bản thuộc label tương ứng

         * count(c): tổng số lượng word trong văn bản thuộc label tương ứng

         *|V|: số lượng từ khác nhau trên label tương ứng

          * +1 ở đây để tránh trường hợp khi predict một title mà có từ chưa có trong tập training sẽ dẫn đến xác suất có điều kiện của cả câu sẽ bằng 0 => gây sai lệch tròn dự đoán

Đánh giá hiệu suất model bằng accuracy_score: là thương của tổng số lượng các mẫu được dự đoán chính xác với số lượng mẫu dự đoán 
   

In [ ]:
y_model=model.predict(X_validation)
print(accuracy_score(y_validation,y_model)) 

Với model này thì ta thấy được có khoảng 89% nhãn được dự đoán đúng. Dù hiệu suất chưa cao nhưng không đến nỗi tệ

**Fine Turning**

Sử dụng Grid Search từ thư viện sklearn: đánh giá hiệu suất model dựa trên các hyperparameter được cung cấp để chọn ra các hyperparameter mà làm cho model đạt hiệu suất cao nhất có thể


In [ ]:
from sklearn.model_selection import GridSearchCV  
#các hyperparameter cần đánh giá để đưa ra lựa chọn tốt nhất
parameters={ 'vect__ngram_range':[(1,1),(1,2),(2,2)], #chọn lựa thuộc tính ngra,_range của hàm CountVectorizer() được xây dựng ở pipeline model, [1,1] đếm theo từ đơn, [1,2] đếm theo từ đơn và theo từng cặp,[2,2] chỉ đếm theo cặp
             'tfidf__use_idf': (True,False),  #chọn lựa thuộc tính use_idf của hàm TfidfTransformer(), nếu True thì có thực hiện phương pháp idf giảm giá trị của các từ gây nhiễu, False thì ngược lại     
             'clf__alpha':(1e-2,1e-3,1), #Laplace smoothing: tham số được thêm vào lúc tính xác suất có điều kiện ở tử số để tránh trường hợp nếu từ đó không có trong bag of words thì gây ra sai lệch
           }
gs_clf=GridSearchCV(model,parameters,n_jobs=-1) 
#GridSearch trên estimator=model,tham số grid là danh sách các tham số parameters ở trên 
#n_job=-1 => Sử dụng tất cả CPU để xử lý => Tăng tốc độ xử lý, xử lý
gs_clf.fit(X_train,y_train)  # tiến hành fit trên training dataset để tìm ra tham số tốt nhất cho model

.best_params => xem tham số nào được chọn để model đạt hiệu suất tốt nhất

In [ ]:
gs_clf.best_params_

Với kết quả trên ta thấy laplace smoothing được lựa chọn bằng 1, có sử dụng idf và xây dựng bag of words dựa trên đếm cả từ đơn và từ đôi là đạt accuracy cao nhất

Train lại model với tham số mới: 


In [ ]:
hyper_model=Pipeline([('vect',CountVectorizer(ngram_range=(1,2))),     #stop_word='english' => loại bỏ các từ thuộc stop_word trong tiếng Anh ( the,on,a,an,...) 
               ('tfidf',TfidfTransformer(use_idf=True)),
               ('clf',MultinomialNB(alpha=1.0)),
])
hyper_model.fit(X_train,y_train)

In [ ]:
new_y_model=hyper_model.predict(X_validation)
print("Old model accuracy: ",accuracy_score(y_validation,y_model))
print("New model accuracy: ",accuracy_score(y_validation,new_y_model))

Ta thấy được sự cải thiện rõ rệt sau khi Fine Tuning: tỉ lệ dự đoán chính xác tăng 4%

Cách sử dụng model đã train: Gọi hàm predict_category với tham số truyền vào là một câu nói bất kỳ

In [ ]:
def predict_category(s,train=X_train,model=hyper_model):
   p=[s]                             #vì một câu sẽ là một vector nên ta phải chuyển dạng chuỗi thành vector
   pred = model.predict(p)           #tiến hành dự đoán
   #giá trị trả về sẽ là 0 hoặc 1, 1 là is_sarcasm, 0 là non_sarcasm
   if (pred==1): print("is_sacarsm")  
   else: print("non_sarcasm")

VD:

In [ ]:
predict_category("11 Simple Cocktail Recipes To Try While Quarantining")

Đối chiếu performance trên 2000 Headlines mới

In [ ]:
X_new=collected_data['headline'].str.replace('\d+', '')
newheadline_y_model=hyper_model.predict(X_new)
accuracy_score(collected_data['is_sarcastic'],newheadline_y_model)

Hiệu suất sụt giảm nghiêm trọng so với bộ dữ liệu cũ vì có những tin tức mới hay nhiều word mới có giá trị lớn liên quan đến bài toán không có trong bag of words => sai lệch => giảm hiệu suất

Nhận xét về nhóm bài toán này: Đây là bài toán khó, không thể đạt độ chính xác cao nếu chỉ sử dụng machine learning vì tin tức mỗi ngày mỗi khác, mỗi đổi mới nếu không cập nhật các tin tức mà chỉ sử dụng các bag of words cũ đã được thu thập từ trước thì sẽ thiếu hụt các từ có giá trị cao(weight). Giải pháp: nên sử dụng deep reinforcement learning để giải quyết bài toán này 

Thành viên nhóm: 18520017 Lê Phước Đạt

Link bộ dữ liệu gốc và tự thu thập: https://drive.google.com/drive/u/0/folders/1sIGYDYuKDfSrkSF9VvVg-EIoy0rdqqsB

Link code mẫu: https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a